In [2]:
# XGBoost was designed to be used with large and complicated datasets.

***XGBoost (Regression):***

<ol>
<li><p>The very first step in fitting XGBoost to the training data is to make an intital prediction. This prediction can be anything, but by default it is 0.5, regardless of whether you are using XGBoost for regression or classification<br><img src='a.png' width=500><br>Note that the black thick line is the predicted value (0.5)</p></li>
<li><p>Now, just like gradient boosting, XGBoost also create decision trees which fits on the residuals. But unlike gradient boosting which uses general decision trees, XGBoost uses unique decision tree to fit the residuals.</p></li>
<li><p>There are many ways to bulid this unique tree or say XGBoost Tree, but for now, we will focus on buliding XGBoost Tree for regression.
<ol>
<li><p>Each tree starts with a single leaf and all of the residuals go onto the same leaf. Now we calculate a Quality Score or a Similarity Score for the residuals. <br><img src='b.png' width=500><br>In the formula, lambda is a regularization parameter. Since our formula says that the sum of the residuals is squared, we can conclude that if there are residuals like +x and -x, then the sum of the residuals will be zero. </p></li>
<li><p>Now we take the least two independent features and avg them. Now we divide the tree into two nodes, if the independent features are less than the avg value, it will go to one of the nodes and the independent features that are greater than the avg value will go to the other node. Note that the residuals of the independent features go to the node and not the value of the independent feature. Now we calculate the similarity score for those nodes. The picture for this step would be similar to the one below.<br><img src='c.png' width=500><br></p></li>
<li><p>Note that if the residuals are similar i.e., both are either positive or negative then we get a pretty high similarity score and if they are different i.e., one of them is positive and the other one is negative, then the similarity score is relatively low.</p></li>
<li><p>Now we need to quantify how much better the leaves cluster similar Residuals than the root. We do this using a formula called 'Gain'. Gain is the sum of the similarity scores for the leaves on the left and the right minus the similarity score for the root.<br><img src='d.png'><br>So that now we have calculated the gain for the first treshold (i.e., the avg value of the least two independent features), we calculate the same for the next treshold (i.e., the avg for the next two least nodes (second last and third last)). Then we compare the similarity scores for the two. The one with the large Gain is meant to be better at splitting the residuals into cluster of similar values.</p></li>
<li><p>Now we repeat the process (we repeat the process till you can split the independent features into two parts) of changing the treshold, calculating the similarity score and then gain, and finally comparing the value of Gain. Note that our depth of the tree is One in every case.</p></li>
<li><p>Now, we will use the treshold that gave us the largest gain as the first branch of our tree. <br><img src='e.png'><br> Now we will repeat the same procedure for the child nodes. Since, the first child node has only one value, we cannot split it further. Now we go to the second leaf node. We split this node using the treshold as the avg of least two values and then calculate the similarity score and then the gain. We repeat this step with various tresholds and take the branch with the largest gain and put that as the leaf nodes for the initial leaf node (i.e., you repeat the procedure ( that you have done for the first branch ) on the leaf node). Similarly you create all the child nodes.</p></li>
</ol>
</p></li>
<li><p>Now we need to prune this tree (pruning is a technique to handle the overfitting of the decision tree). We prune an XGTree based on its Gain Values.<br><ol>
<li><p>First we assign a value to a varaible called Gamma. Then, we take the lowest branch (or the child node previous to the leaf node) and then subtract its Gain with the gamma.</p></li>
<li><p>If the value is negative, we remove the branch.<br><img src='f.png' width=600><br></p></li>
<li><p>If the value is positive we will not remove that branch. Note: There are cases when the subtraction may be negative for the higher nodes or root node but since the lower branches have the gain larger than the gamma (i.e., the subtracted value is positive), we will not remove the higher branches or the root node.<br><img src='g.png' width=700><br> But if the value of gamma is higher than all the gains, then the decision tree is removed and you are only left with the initial prediction (i.e., 0.5)</p></li>
</ol></p></li>
</ol>

<p>Remember, lambda is a regularization parameter, which means that it is intended to reduce the prediction's sensitivity to individual observations.</p>

 When Lambda is greater than zero, it is easier to prune/remove leaves as the values of gain is smaller

Sometimes you can also get the values of Gain as negative. For these cases, even if your gamma is zero, the branch gets pruned. Therefore, setting the value of gamma to zero does not turn off pruning.

Setting lambda value greater than or equal to zero prevents overfitting of the training data.

<p>Now that we have pruned our XGTree, we calculate the output values for each leaf node. Output value is the sum of residuals of each node divided by the sum of number of residuals and the regularization parameter lambda.<br><img src='h.png'><br><img src='i.png'><br> Now, our first tree is complete.</p>

Since, we built our first tree, we can make new predictions. Just like graient boosting, XGBoost also uses the initial prediction (0.5) and then add it to the product of learning rate (eta) and the output values for the respective independent features.<br><img src='j.png' width=700><br><img src='k.png' width=700><br>You will observe that since we are adding the output value to the initial prediction, we are getting close to the original value. Now we build a decision tree with the new residuals and make predictions that give us even smaller residuals. We continue to build these trees until it reaches the original values.

***XGBoost for Classification:***

Consider the classification to be like the one below:<br><img src='l.png' width=300><br>The very first step to train the XGBoost is to make the initial prediction (0.5). We take 0.5 because we assume that there are 50% chances of our data to be in the first category or the second category. We can illustrate the initial prediction by adding a y-axis to the graph to represent the probability that the drug is effective.<br><img src='m.png' width=500><br>Now, just like we did for regression, we fit in XGBoost trees to the residuals. However the formula for the similarity score will be changed.<br><img src='n.png' width=500><br>The faded part is also in the formula. We bulid the decision tree by first taking a treshold (avg value for the last two independent features) and then splitting the independent features into two leaves. Then , we calculate the similarity score for each node. The previous probability in the formula refers to the residual line (0.5 since it is our first tree). Now we calculate the gain by using the same formula i.e., the sum of the similarity scores of the leaf nodes subtracted with the similarity score of the root node. Now, we change the treholds and consider the tree with the largest gain and make it as the first branch of our tree. Similarly we will create the other branches. 

***Note:***<br>Wondering what would be the depth of an XGBoost Tree (either regression/classification). Well the depth of an XGBoost Tree depends on something called Cover. Cover for an XGBoost tree is the [(denominator of the similarity score)-lambda]. Since the default value of the cover is One, the regression XGBoost tree can have a minimum depth of One. Hence, cover has no effect on Regression XGBoost Tree. But in the case of classification, if the cover of a node is less than the default value i.e., One, then XGBoost will not allow you to build that node.

Since, we have created our XGBoost Tree, we must now prune the tree. Just like we prune the Regression XGBoost Tree using the formula (Gain-gamma) and check whether the subtracted value is positive or negative, we do similar thing in this case.

Now that our tree is pruned, we find out the Output values for each leaf node.<br><img src='o.png' width=500><br>Now that we have build our first tree, its time to make some predictions.<br><img src='p.png' width=500><br>

Formula for prediction is similar to that of regression but in this case, instead of using initial value (0.5) we use log of odds for that initial value i.e., log(p/(1-p)) where p=initial value and the final value is the log of odds of the prediction. So, log(odds) prediction=log of odds + learning_rate*(output values of the respective independent features).
Now, we convert the log(odds) to probability, we use this formula:<br><img src='q.png' width=500><br>

The above formula is called a Logistic Function. Now we plot this probability on the y-axis for this independent feature. You will observe that our line becomes closer to the original value.<br><img src='r.png' width=500><br>

We repeat this process for all the output values and then make a new decision tree using these output values. But in this case, the initial predicition will be the previous probability of the independent feature (while using the formula for prediciton, similarity score and output value). We repeat this procedure until we reach the maximum no. of trees (if mentioned) or if the residuals get super small.

In [ ]:
# First, you need to install xgboost module. (pip install xgboost)
# import xgboost
# clf=xgboost.XGBClassifier()